In [1]:
##  Import libraries

import pandas as pd, numpy as np
nate_silver = pd.read_csv(r'C:\Users\benca\Downloads\nfl-elo\nfl-elo\nfl_elo.csv')
odds = pd.read_csv(r'C:\Users\benca\Downloads\NFL Odds Data - NFL Odds.csv')
pd.set_option("display.max_columns", 100)

## Calculates implied odds of a given moneyline
## What I call implied odds- appears to not be what most gamblers call implied odds-
## This function returns the frequency with which you must win a bet in order to break even

def imp(mline):
    global implied
    if mline > 0:
        implied = 100/(mline + 100)
    if mline < 0:
        implied = abs(mline)/(100 + abs(mline))
    return implied 

## Calculates Decimal Odds from Moneyline
## imp(z) = 1/to_dec(z)

def to_dec(mline):
    if mline > 0:
        return (mline/100 + 1)
    if mline < 0:
        return (100/abs(mline) + 1)
    
## Calculates Expected Value from a given moneyline, "real" probability, and betsize

def ev(mline, model, betsize=1):
    global implied
    if mline > 0:
        implied = 100/(mline + 100)
    if mline < 0:
        implied = abs(mline)/(100 + abs(mline))
    return (model * betsize * ((1/implied) - 1)) + ((model-1) * betsize)

## Calculates actual return from Decimal Odds and Result of Game

def av(dec, result):
    global send
    if result == 'W':
        send = dec-1
    if result == 'L':
        send = -1
    else: send = 0
    return send

In [2]:
model = nate_silver[nate_silver['season'] > 2006].reset_index(drop=True)
model['id'] = model.index + 1

In [3]:
new_date = pd.DataFrame(data=[], columns=['new_date'])

for n in odds.index:
    if odds['Date'][n] < 1000:
        temp = pd.DataFrame(data=[[(str(odds['Year'][n]) + '-0' +  str(odds['Date'][n])[0] + "-" + str(odds['Date'][n])[1:3])]], columns=['new_date'])
    else: temp = pd.DataFrame(data=[[(str(odds['Year'][n]) + '-' + str(odds['Date'][n])[0:2] + "-" + str(odds['Date'][n])[2:4])]], columns=['new_date'])
    
    new_date = pd.concat([new_date,temp], ignore_index=True)

odds = pd.concat([odds,new_date],axis=1)
        
teamNames = {'NewOrleans':'NO', 'Indianapolis':'IND', 'KansasCity':'KC', 'HoustonTexans':'HOU',
       'Denver':'DEN', 'Buffalo':'BUF', 'Pittsburgh':'PIT', 'Cleveland':'CLE', 'Tennessee':'TEN',
       'Jacksonville':'JAX', 'Carolina':'CAR', 'St.Louis':'LAR', 'Philadelphia':'PHI', 'GreenBay':'GB',
       'Atlanta':'ATL', 'Minnesota':'MIN', 'Miami':'MIA', 'Washington':'WSH', 'NewEngland':'NE',
       'NYJets':'NYJ', 'TampaBay':'TB', 'Seattle':'SEA', 'Chicago':'CHI', 'SanDiego':'LAC', 'Detroit':'DET',
       'Oakland':'OAK', 'NYGiants':'NYG', 'Dallas':'DAL', 'Baltimore':'BAL', 'Cincinnati':'CIN',
       'Arizona':'ARI', 'SanFrancisco':'SF', 'Houston':'HOU', 'BuffaloBills':'BUF', 'NewYork':'NYG',
       'LosAngeles':'LAR', 'LARams':'LAR', 'LAChargers':'LAC', 'LasVegas':'OAK', 'LVRaiders':'OAK',
       'KCChiefs':'KC', 'Kansas':'KC', 'Tampa':'TB', 'Washingtom':'WSH'}

odds = odds.replace(to_replace = teamNames)

In [4]:
odds['new_date'][2052] = '2010-12-25'
odds['new_date'][2053] = '2010-12-25'
odds['new_date'][3858] = '2014-09-29'
odds['new_date'][3859] = '2014-09-29'
odds['new_date'][4428:4452] = '2015-10-18'
odds['new_date'][4800] = '2016-01-24'
odds['new_date'][4801] = '2016-01-24'
odds['new_date'][4802] = '2016-01-24'
odds['new_date'][4803] = '2016-01-24'

<ipython-input-4-bd3381fb76a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_date'][2052] = '2010-12-25'
<ipython-input-4-bd3381fb76a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_date'][2053] = '2010-12-25'
<ipython-input-4-bd3381fb76a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_date'][3858] = '2014-09-29'
<ipython-input-4-bd3381fb76a9>:4: SettingWithCopyWarning: 
A value is trying to be se

In [5]:
## Using the new matching dates and names
## Assign the unique game ID from the 538 data to the odds data

odds['id'] = 0
for n in model.index:
    temp = odds[odds['new_date'] == model['date'][n]]
    temp = temp[(temp['Team'] == model['team1'][n]) | (temp['Team'] == model['team2'][n])]
    for x in temp.index:
        odds['id'][x] = n + 1

<ipython-input-5-fc0799460650>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['id'][x] = n + 1


In [6]:
temp = model.rename(columns={
        'team1':'team',
        'team2':'opponent',
        'elo1_pre':'elo_pre',
        'elo2_pre':'elo_pre_opp',
        'elo_prob1':'elo_prob',
        'elo_prob2':'elo_prob_opp', 
        'elo1_post':'elo_post',
        'elo2_post':'elo_post_opp',
        'qbelo1_pre':'qb_elo_pre',
        'qbelo2_pre':'qb_elo_pre_opp', 
        'qb1':'qb', 
        'qb2':'qb_opp', 
        'qb1_value_pre':'qb_value_pre',
        'qb2_value_pre':'qb_value_pre_opp',
        'qb1_adj':'qb_adj',
        'qb2_adj':'qb_adj_opp', 
        'qbelo_prob1':'qbelo_prob',
        'qbelo_prob2':'qbelo_prob_opp',
        'qb1_game_value':'qb_game_value',
        'qb2_game_value':'qb_game_value_opp', 
        'qb1_value_post':'qb_value_post',
        'qb2_value_post':'qb1_value_post_opp',
        'qbelo1_post':'qbelo_post',
        'qbelo2_post':'qbelo_post_opp', 
        'score1':'score', 
        'score2':'score_opp'})
temp2 = model.rename(columns={
        'team2':'team',
        'team1':'opponent',
        'elo2_pre':'elo_pre',
        'elo1_pre':'elo_pre_opp',
        'elo_prob2':'elo_prob',
        'elo_prob1':'elo_prob_opp', 
        'elo2_post':'elo_post',
        'elo1_post':'elo_post_opp',
        'qbelo2_pre':'qb_elo_pre',
        'qbelo1_pre':'qb_elo_pre_opp', 
        'qb2':'qb', 
        'qb1':'qb_opp', 
        'qb2_value_pre':'qb_value_pre',
        'qb1_value_pre':'qb_value_pre_opp',
        'qb2_adj':'qb_adj',
        'qb1_adj':'qb_adj_opp', 
        'qbelo_prob2':'qbelo_prob',
        'qbelo_prob1':'qbelo_prob_opp',
        'qb2_game_value':'qb_game_value',
        'qb1_game_value':'qb_game_value_opp', 
        'qb2_value_post':'qb_value_post',
        'qb1_value_post':'qb1_value_post_opp',
        'qbelo2_post':'qbelo_post',
        'qbelo1_post':'qbelo_post_opp', 
        'score2':'score', 
        'score1':'score_opp'})
test = temp.append(temp2)

In [7]:
test = test.sort_values(by=['id','team'])
test = test.reset_index(drop=True)
odds = odds.sort_values(by=['id','Team'])
odds = odds.reset_index(drop=True)
nfl = pd.concat([test,odds],axis=1)
## deleting the duplicate ID column
nfl = nfl.loc[:,~nfl.columns.duplicated()]

In [8]:
win = pd.DataFrame(data=[],columns=['win'])
for x in nfl.index:
    if nfl['score'][x] > nfl['score_opp'][x]:
        temp = pd.DataFrame(data=['W'], columns = ['win'])
    elif nfl['score'][x] == nfl['score_opp'][x]:
        temp = pd.DataFrame(data=['T'], columns = ['win'])
    else: temp = pd.DataFrame(data=['L'], columns = ['win'])
    
    win = pd.concat([win,temp],ignore_index=True)

nfl = pd.concat([nfl,win],axis=1)

In [9]:
## Calculating Decimal odds
nfl['dec'] = nfl['ML'].apply(to_dec, convert_dtype=True)
## Creating Expected Value from the QB ELO probabilities
nfl['ev_elo'] = np.vectorize(ev)(nfl['ML'], nfl['elo_prob'])
nfl['ev_qbelo'] = np.vectorize(ev)(nfl['ML'], nfl['qbelo_prob'])
nfl['elo_prob_avg'] = (nfl['elo_prob'] + nfl['qbelo_prob'])/2
nfl['ev_elo_avg'] = np.vectorize(ev)(nfl['ML'], nfl['elo_prob_avg'])

In [10]:
## Calculating ACTUAL VALUE / ACTUAL RETURN
## From decimal odds and game results
av = pd.DataFrame(data=[],columns=['av'])

for z in nfl.index:
    
    if (nfl['win'][z] == 'W'):
        temp = pd.DataFrame(data=[[nfl['dec'][z] - 1.0]],columns=['av'])
    elif (nfl['win'][z] == 'L'):
        temp = pd.DataFrame(data=[[-1]],columns=['av'])
    else: 
        temp = pd.DataFrame(data=[[0]],columns=['av'])
        
    av = pd.concat([av,temp], ignore_index=True)
    
nfl = pd.concat([nfl,av], axis=1)

In [35]:
nfl.to_csv('nfl_data.csv')

In [36]:
nfl

,date,season,neutral,playoff,team,opponent,elo_pre,elo_pre_opp,elo_prob,elo_prob_opp,elo_post,elo_post_opp,qb_elo_pre,qb_elo_pre_opp,qb,qb_opp,qb_value_pre,qb_value_pre_opp,qb_adj,qb_adj_opp,qbelo_prob,qbelo_prob_opp,qb_game_value,qb_game_value_opp,qb_value_post,qb1_value_post_opp,qbelo_post,qbelo_post_opp,score,score_opp,id,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Season,Year,new_date,win,dec,ev_elo,ev_qbelo,elo_prob_avg,ev_elo_avg,av
0,2007-09-06,2007,0,NaN,IND,NO,1653.923000,1515.973000,0.762833,0.237167,1668.974000,1500.922000,1615.089436,1528.669157,Peyton Manning,Drew Brees,200.633551,149.485237,11.852944,4.366005,0.705497,0.294503,301.239762,63.756650,210.694172,140.912378,1634.185506,1509.573087,41,10,1,906,302,H,IND,7,3,14,17,41,5.5,5.5,-240,3.5,2007-2008,2007,2007-09-06,W,1.416667,0.080680,-0.000546,0.734165,0.040067,0.416667
1,2007-09-06,2007,0,NaN,NO,IND,1515.973000,1653.923000,0.237167,0.762833,1500.922000,1668.974000,1528.669157,1615.089436,Drew Brees,Peyton Manning,149.485237,200.633551,4.366005,11.852944,0.294503,0.705497,63.756650,301.239762,140.912378,210.694172,1509.573087,1634.185506,10,41,1,906,301,V,NO,0,10,0,0,10,50.5,52.5,200,24.5,2007-2008,2007,2007-09-06,L,3.000000,-0.288499,-0.116492,0.265835,-0.202495,-1.000000
2,2007-09-09,2007,0,NaN,GB,PHI,1497.051000,1542.727000,0.527781,0.472219,1510.030000,1529.748000,1482.197543,1545.374080,Brett Favre,Donovan McNabb,85.291347,150.153166,-3.443745,7.702689,0.476425,0.523575,-42.577230,53.775454,72.504489,140.515395,1496.823119,1530.748504,16,13,2,909,416,H,GB,10,0,3,3,16,43,41.5,160,21,2007-2008,2007,2007-09-09,W,2.600000,0.372230,0.238704,0.502103,0.305467,1.600000
3,2007-09-09,2007,0,NaN,PHI,GB,1542.727000,1497.051000,0.472219,0.527781,1529.748000,1510.030000,1545.374080,1482.197543,Donovan McNabb,Brett Favre,150.153166,85.291347,7.702689,-3.443745,0.523575,0.476425,53.775454,-42.577230,140.515395,72.504489,1530.748504,1496.823119,13,16,2,909,415,V,PHI,0,10,3,0,13,3,3,-190,3.5,2007-2008,2007,2007-09-09,L,1.526316,-0.279244,-0.200859,0.497897,-0.240052,-1.000000
4,2007-09-09,2007,0,NaN,SEA,TB,1511.936000,1417.385000,0.714726,0.285274,1526.342000,1402.979000,1533.217849,1446.007807,Matt Hasselbeck,Jeff Garcia,82.754793,110.546563,-5.369692,13.988117,0.682513,0.317487,161.812315,91.630915,90.660546,108.654999,1549.433314,1429.792342,20,6,3,909,424,H,SEA,0,10,0,10,20,6,6,-275,3,2007-2008,2007,2007-09-09,W,1.363636,-0.025374,-0.069301,0.698619,-0.047337,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,2021-01-24,2020,0,c,TB,GB,1679.186255,1715.623187,0.358032,0.641968,1703.303296,1691.506146,1660.789489,1689.406792,Tom Brady,Aaron Rodgers,221.747091,276.653962,21.701436,31.893640,0.370809,0.629191,121.078453,198.493608,211.680227,268.837927,1684.319058,1665.877222,31,26,3738,124,311,V,TB,7,14,7,3,31,51,53,160,27.5,2020-2021,2021,2021-01-24,W,2.600000,-0.069118,-0.035897,0.364420,-0.052507,1.600000
7476,2021-01-24,2020,0,c,BUF,KC,1719.974149,1719.618921,0.408028,0.591972,1698.505791,1741.087279,1706.159766,1718.032247,Josh Allen,Patrick Mahomes,273.290046,271.760879,36.547695,8.912308,0.464363,0.535637,241.765435,376.765350,270.137585,282.261326,1681.289840,1742.902172,24,38,3739,124,313,V,BUF,9,3,3,9,24,53.5,54.5,140,0.5,2020-2021,2021,2021-01-24,L,2.400000,-0.020734,0.114470,0.436195,0.046868,-1.000000
7477,2021-01-24,2020,0,c,KC,BUF,1719.618921,1719.974149,0.591972,0.408028,1741.087279,1698.505791,1718.032247,1706.159766,Patrick Mahomes,Josh Allen,271.760879,273.290046,8.912308,36.547695,0.535637,0.464363,376.765350,241.765435,282.261326,270.137585,1742.902172,1681.289840,38,24,3739,124,314,H,KC,0,21,10,7,38,4.5,3,-160,27.5,2020-2021,2021,2021-01-24,W,1.625000,-0.038045,-0.129589,0.563805,-0.083817,0.625000
7478,2021-02-07,2020,1,s,KC,TB,1741.087279,1703.303296,0.4

In [16]:
nfl['new_date'][0][0]

'2'

In [13]:
type(nfl['date'][0]

str

In [14]:
nfl.columns

Index(['date', 'season', 'neutral', 'playoff', 'team', 'opponent', 'elo_pre',
       'elo_pre_opp', 'elo_prob', 'elo_prob_opp', 'elo_post', 'elo_post_opp',
       'qb_elo_pre', 'qb_elo_pre_opp', 'qb', 'qb_opp', 'qb_value_pre',
       'qb_value_pre_opp', 'qb_adj', 'qb_adj_opp', 'qbelo_prob',
       'qbelo_prob_opp', 'qb_game_value', 'qb_game_value_opp', 'qb_value_post',
       'qb1_value_post_opp', 'qbelo_post', 'qbelo_post_opp', 'score',
       'score_opp', 'id', 'Date', 'Rot', 'VH', 'Team', '1st', '2nd', '3rd',
       '4th', 'Final', 'Open', 'Close', 'ML', '2H', 'Season', 'Year',
       'new_date', 'win', 'dec', 'ev_elo', 'ev_qbelo', 'elo_prob_avg',
       'ev_elo_avg', 'av'],
      dtype='object')

In [18]:
nfl[['date', 'season', 'neutral', 'playoff', 'team', 'opponent', 'elo_pre',
       'elo_pre_opp', 'elo_prob', 'elo_prob_opp', 
       'qb_elo_pre', 'qb_elo_pre_opp', 'qb', 'qb_opp', 'qb_value_pre',
       'qb_value_pre_opp', 'qb_adj', 'qb_adj_opp', 'qbelo_prob',
       'qbelo_prob_opp', 'id', 'VH','ML', 'Season', 'Year',
       'dec', 'ev_elo', 'ev_qbelo', 'av']]

,date,season,neutral,playoff,team,opponent,elo_pre,elo_pre_opp,elo_prob,elo_prob_opp,qb_elo_pre,qb_elo_pre_opp,qb,qb_opp,qb_value_pre,qb_value_pre_opp,qb_adj,qb_adj_opp,qbelo_prob,qbelo_prob_opp,id,VH,ML,Season,Year,dec,ev_elo,ev_qbelo,av
0,2007-09-06,2007,0,NaN,IND,NO,1653.923000,1515.973000,0.762833,0.237167,1615.089436,1528.669157,Peyton Manning,Drew Brees,200.633551,149.485237,11.852944,4.366005,0.705497,0.294503,1,H,-240,2007-2008,2007,1.416667,0.080680,-0.000546,0.416667
1,2007-09-06,2007,0,NaN,NO,IND,1515.973000,1653.923000,0.237167,0.762833,1528.669157,1615.089436,Drew Brees,Peyton Manning,149.485237,200.633551,4.366005,11.852944,0.294503,0.705497,1,V,200,2007-2008,2007,3.000000,-0.288499,-0.116492,-1.000000
2,2007-09-09,2007,0,NaN,GB,PHI,1497.051000,1542.727000,0.527781,0.472219,1482.197543,1545.374080,Brett Favre,Donovan McNabb,85.291347,150.153166,-3.443745,7.702689,0.476425,0.523575,2,H,160,2007-2008,2007,2.600000,0.372230,0.238704,1.600000
3,2007-09-09,2007,0,NaN,PHI,GB,1542.727000,1497.051000,0.472219,0.527781,1545.374080,1482.197543,Donovan McNabb,Brett Favre,150.153166,85.291347,7.702689,-3.443745,0.523575,0.476425,2,V,-190,2007-2008,2007,1.526316,-0.279244,-0.200859,-1.000000
4,2007-09-09,2007,0,NaN,SEA,TB,1511.936000,1417.385000,0.714726,0.285274,1533.217849,1446.007807,Matt Hasselbeck,Jeff Garcia,82.754793,110.546563,-5.369692,13.988117,0.682513,0.317487,3,H,-275,2007-2008,2007,1.363636,-0.025374,-0.069301,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,2021-01-24,2020,0,c,TB,GB,1679.186255,1715.623187,0.358032,0.641968,1660.789489,1689.406792,Tom Brady,Aaron Rodgers,221.747091,276.653962,21.701436,31.893640,0.370809,0.629191,3738,V,160,2020-2021,2021,2.600000,-0.069118,-0.035897,1.600000
7476,2021-01-24,2020,0,c,BUF,KC,1719.974149,1719.618921,0.408028,0.591972,1706.159766,1718.032247,Josh Allen,Patrick Mahomes,273.290046,271.760879,36.547695,8.912308,0.464363,0.535637,3739,V,140,2020-2021,2021,2.400000,-0.020734,0.114470,-1.000000
7477,2021-01-24,2020,0,c,KC,BUF,1719.618921,1719.974149,0.591972,0.408028,1718.032247,1706.159766,Patrick Mahomes,Josh Allen,271.760879,273.290046,8.912308,36.547695,0.535637,0.464363,3739,H,-160,2020-2021,2021,1.625000,-0.038045,-0.129589,0.625000
7478,2021-02-07,2020,1,s,KC,TB,1741.087279,1703.303296,0.460913,0.539087,1742.902172,1684.319058,Patrick Mahomes,Tom Brady,282.261326,211.680227,13.716916,15.582933,0.533794,0.466206,3740,N,-150,2020-2021,2021,1.666667,-0.231812,-0.110343,-1.000000


In [ ]:
'elo_post', 'elo_post_opp','qb_game_value', 'qb_game_value_opp', 'qb_value_post',
       'qb1_value_post_opp', 'qbelo_post', 'qbelo_post_opp', 'score',
       'score_opp','Team', '1st', '2nd', '3rd',
       '4th', 'Final','Date', 'Rot','Open', 'Close', '2H', 'new_date', 'win','elo_prob_avg',
       'ev_elo_avg',